# Infonet Security HW2
#### Harris A. Ransom
#### 10/09/2024

## Imports

In [44]:
# Imports
from datetime import datetime
from math import gcd, sqrt
import random
import numpy as np

## 1. One-Time Pad 

In [45]:
# One-Time Pad XOR Encrypt Function
def otpEncrypt(m, k):
	ciphertext = []
	if (len(m) != len(k)):
		raise ValueError("Message and key are not the same length")
	
	for i in range(0, len(m)):
		c = ord(m[i]) ^ ord(k[i])
		ciphertext.append(chr(c))
	return ciphertext

# One-Time Pad XOR Decrypt Function
def otpDecrypt(c, k):
	plaintext = []
	if (len(c) != len(k)):
		raise ValueError("Ciphertext and key are not the same length")

	for i in range(0, len(c)):
		p = ord(c[i]) ^ ord(k[i])
		plaintext.append(chr(p))
	return "".join(plaintext)

# Initialize plaintext and key
plaintext = "helloAlice!"
key = "$%wB+=?Qz?4" # I know defining a key in plaintext is bad practice, but it's just for testing purposes

# Encrypt plaintext
ciphertext = otpEncrypt(plaintext, key)
print(f"Ciphertext: {ciphertext}")

# Decrypt ciphertext
decrypted = otpDecrypt(ciphertext, key)
print(f"Decrypted Message: {decrypted}")
if (decrypted == plaintext):
	print("Plaintext successfully encrypted and decrypted!")
else:
	print("Error in encryption/decryption process!")

Ciphertext: ['L', '@', '\x1b', '.', 'D', '|', 'S', '8', '\x19', 'Z', '\x15']
Decrypted Message: helloAlice!
Plaintext successfully encrypted and decrypted!


## 2. Blum-Blum-Shub PRNG

In [46]:
# Helper function to check if a number is prime
def isPrime(n):
    prime_flag = 0

    if(n > 1):
        for i in range(2, int(sqrt(n)) + 1):
            if (n % i == 0):
                prime_flag = 1
                break
        if (prime_flag == 0):
            return True
        else:
            return False
    else:
        return False
    
# Helper function to check if two numbers are relatively prime
def coprime(a, b):
    return gcd(a, b) == 1

# Generates N pseudo-random bits
def bbsGenerate(p, q, seed, numBits):
	prngBits = []
	x = []
	n = p*q

	# Initialize with seed value
	x0 = (seed**2) % n
	x.append(x0)
	prngBits.append(x0 % 2)

	# Generate rest of the bits
	for i in range(0, numBits-1):
		xi = (x[i] ** 2) % n
		x.append(xi)
		prngBits.append(xi % 2)
	return prngBits

# Choose p, q, seed
# p = q = 3 (mod 4)
p = (random.randint(100000000, 10000000000) * 4) + 3
if (not isPrime(p)):
    p = (random.randint(100000000, 10000000000) * 4) + 3
q = (random.randint(100000000, 10000000000) * 4) + 3
if (not isPrime(q)):
    q = (random.randint(100000000, 10000000000) * 4) + 3
seed = int(((datetime.now() - datetime(1970, 1, 1)).total_seconds()) * (10**6)) # Unix timestamp
n = p*q
while (not coprime(n, seed)):
	seed = int(((datetime.now() - datetime(1970, 1, 1)).total_seconds()) * (10**6))

# Generate N pseudo-random bits
prngBits = bbsGenerate(p, q, seed, 10**6)
#print(prngBits)

# Check generated bits for 0s and (0,0) pairs
zeroCount = 0
zeroPairCount = 0
for i in range(0, len(prngBits) - 1):
	if (prngBits[i] == 0):
		zeroCount += 1
		if (prngBits[i + 1] == 0):
			zeroPairCount += 1
if (prngBits[-1] == 0):
	zeroCount += 1
print(f"Number of 0s: {zeroCount}")
print(f"Number of (0,0) pairs: {zeroPairCount}")
freqZeros = zeroCount / len(prngBits)
freqZeroPairs = zeroPairCount / (len(prngBits) - 1)
print(f"Frequency of 0s: {freqZeros}")
print(f"Frequency of (0,0) pairs: {freqZeroPairs}")

Number of 0s: 499077
Number of (0,0) pairs: 248849
Frequency of 0s: 0.499077
Frequency of (0,0) pairs: 0.24884924884924886


## 3. Linear Feedback Shift Register (LFSR)

Given recurrence:
$$
    x_i = x_{i-4} + x_{i-10} + x_{i-11} + x_{i-12} (\text{mod} 2)
$$

<ol type="a">
  <li>This recurrence has a length of m=12. The maximal length before this recurrence repeats itself is $2^{12} - 1 = 4095$.</li>
  <li>12 bits are needed to initialize this LFSR.</li>
</ol>

In [47]:
# LFSR Implementation (c)
def LFSR(initialBits, N, table=False):
	if (len(initialBits) != 12):
		raise ValueError("Initial bits must be 12 bits long")
	
	# Initialize LFSR
	state = initialBits

	# Calculate random bits
	# Coefficients (n-i): 4, 10, 11, 12
	outputBits = []
	if (table): 
		print("State: \t \t \t \t Output:")
	for i in range(0, N):
		sum = (state[0] + state[1] + state[2] + state[8]) % 2
		state[0:11] = state[1:12]
		state[11] = sum
		outputBits.append(sum)
		if (table):
			print(f"{state} \t {sum}")
	return outputBits

# Generate bits with LFSR
#initialBits = np.array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]) # For testing, this is statically set
initialBits = np.random.choice([0,1], 12)
while (not initialBits.any()):
    initialBits = np.random.choice([0,1], 12)
print(f"Initialization Vector: {initialBits}")
N = 4095
randomBits = LFSR(initialBits, N)

# Calculate 0 and (0, 0) frequencies (d)
zeroCount = 0
zeroPairCount = 0
for i in range(0, len(randomBits) - 1):
	if (randomBits[i] == 0):
		zeroCount += 1
		if (randomBits[i + 1] == 0):
			zeroPairCount += 1
if (randomBits[-1] == 0):
	zeroCount += 1
print(f"Number of 0s: {zeroCount}")
print(f"Number of (0,0) pairs: {zeroPairCount}")
freqZeros = zeroCount / len(randomBits)
freqZeroPairs = zeroPairCount / (len(randomBits) - 1)
print(f"Frequency of 0s: {freqZeros}")
print(f"Frequency of (0,0) pairs: {freqZeroPairs}")

Initialization Vector: [1 1 1 0 0 1 1 0 1 0 1 1]
Number of 0s: 2047
Number of (0,0) pairs: 1023
Frequency of 0s: 0.4998778998778999
Frequency of (0,0) pairs: 0.24987787005373718


## 4. LFSR cont.

Given recurrence: 
$$
x_i = c_i x_{i-1} + ... + c_m x_{i-m} (\text{mod} 2)
$$

We can show that, if the number of non-zero coefficients is odd, then the recurrence will not generate a maximal-length sequence. TODO